In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
from main_agent import Agent
import os
import requests
import pandas as pd
import tempfile

In [38]:
DEFAULT_API_URL = "https://agents-course-unit4-scoring.hf.space"
api_url = DEFAULT_API_URL
questions_url = f"{api_url}/questions"
submit_url = f"{api_url}/submit"
files_url = f"{api_url}/files"

response = requests.get(questions_url, timeout=15)
response.raise_for_status()
questions_data = response.json()

for i, item in enumerate(questions_data):

    process = [11]
    if i not in process:
        continue

    task_id = item.get("task_id")
    question_text = item.get("question")

In [42]:
task_id

'f918266a-b3e0-4914-865d-4faa564f1aef'

In [39]:
question_text

'What is the final numeric output from the attached Python code?'

In [3]:
# get question

def test_run():

    space_id = "andrea-giorni/agents-course-unit4-basic-agent"

    DEFAULT_API_URL = "https://agents-course-unit4-scoring.hf.space"
    api_url = DEFAULT_API_URL
    questions_url = f"{api_url}/questions"
    submit_url = f"{api_url}/submit"
    files_url = f"{api_url}/files"

    # 1. Instantiate Agent ( modify this part to create your agent)
    try:
        agent = Agent()
    except Exception as e:
        print(f"Error instantiating agent: {e}")
        return f"Error initializing agent: {e}", None
    # In the case of an app running as a hugging Face space, this link points toward your codebase (usefull for others so please keep it public)
    agent_code = f"https://huggingface.co/spaces/{space_id}/tree/main"
    print(agent_code)

    # 2. Fetch Questions
    print(f"Fetching questions from: {questions_url}")
    try:
        response = requests.get(questions_url, timeout=15)
        response.raise_for_status()
        questions_data = response.json()
        if not questions_data:
            print("Fetched questions list is empty.")
            return "Fetched questions list is empty or invalid format.", None
        print(f"Fetched {len(questions_data)} questions.")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching questions: {e}")
        return f"Error fetching questions: {e}", None
    except requests.exceptions.JSONDecodeError as e:
        print(f"Error decoding JSON response from questions endpoint: {e}")
        print(f"Response text: {response.text[:500]}")
        return f"Error decoding server response for questions: {e}", None
    except Exception as e:
        print(f"An unexpected error occurred fetching questions: {e}")
        return f"An unexpected error occurred fetching questions: {e}", None

    # 3. Run your Agent
    results_log = []
    answers_payload = []
    print(f"Running agent on {len(questions_data)} questions...")
    for i, item in enumerate(questions_data):

        process = [11]
        if i not in process:
            continue

        task_id = item.get("task_id")
        question_text = item.get("question")
        if not task_id or question_text is None:
            print(f"Skipping item with missing task_id or question: {item}")
            continue
        try:
            file_path: Optional[str] = None
            try:
                file_response = requests.get(f"{files_url}/{task_id}", timeout=15)
                if file_response.status_code == 200 and file_response.content:
                    # Get filename from Content-Disposition header or URL
                    filename = None
                    content_disposition = file_response.headers.get(
                        "Content-Disposition"
                    )
                    if content_disposition and "filename=" in content_disposition:
                        filename = content_disposition.split("filename=")[-1].strip('"')
                    else:
                        # Try to get filename from URL
                        url = file_response.url
                        filename = url.split("/")[-1]
                        if not filename or filename == str(task_id):
                            filename = f"file_{task_id}"

                    # Create temp directory and save file with original name
                    temp_dir = tempfile.mkdtemp()
                    file_path = os.path.join(temp_dir, filename)
                    with open(file_path, "wb") as f:
                        f.write(file_response.content)
                    print(f"Downloaded file for task {task_id} to {file_path}")
                else:
                    print(f"No file for task {task_id} or file is empty.")
            except Exception as e:
                print(f"Error downloading file for task {task_id}: {e}")
                file_path = None

            submitted_answer = agent.builder.invoke(
                input={'messages': question_text,
                'file_path': file_path}
                )
            answers_payload.append(
                {"task_id": task_id, "submitted_answer": submitted_answer}
            )
            results_log.append(
                {
                    "Task ID": task_id,
                    "Question": question_text,
                    "Submitted Answer": submitted_answer,
                }
            )
        except Exception as e:
            print(f"Error running agent on task {task_id}: {e}")
            results_log.append(
                {
                    "Task ID": task_id,
                    "Question": question_text,
                    "Submitted Answer": f"AGENT ERROR: {e}",
                }
            )

    if not answers_payload:
        print("Agent did not produce any answers to submit.")
        return "Agent did not produce any answers to submit.", pd.DataFrame(results_log)
    return results_log
    

In [4]:
test_run()

https://huggingface.co/spaces/andrea-giorni/agents-course-unit4-basic-agent/tree/main
Fetching questions from: https://agents-course-unit4-scoring.hf.space/questions
Fetched 20 questions.
Running agent on 20 questions...
Downloaded file for task f918266a-b3e0-4914-865d-4faa564f1aef to C:\Users\ANDREA~1.GIO\AppData\Local\Temp\tmp1qjn3jhf\f918266a-b3e0-4914-865d-4faa564f1aef.py


[{'Task ID': 'f918266a-b3e0-4914-865d-4faa564f1aef',
  'Question': 'What is the final numeric output from the attached Python code?',
  'Submitted Answer': {'messages': [HumanMessage(content='What is the final numeric output from the attached Python code?', additional_kwargs={}, response_metadata={}, id='a6678c53-d33c-4b9b-8755-dda03d677ffe'),
    SystemMessage(content='File path provided: C:\\Users\\ANDREA~1.GIO\\AppData\\Local\\Temp\\tmp1qjn3jhf\\f918266a-b3e0-4914-865d-4faa564f1aef.py', additional_kwargs={}, response_metadata={}, id='96e9d09b-83fc-4b4b-af72-358ac7e82595'),
    AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jdSq4awjVLISHfg2M2fVgKZK', 'function': {'arguments': '{"script_path":"C:\\\\Users\\\\ANDREA~1.GIO\\\\AppData\\\\Local\\\\Temp\\\\tmp1qjn3jhf\\\\f918266a-b3e0-4914-865d-4faa564f1aef.py"}', 'name': 'python-script-opener'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 433, '

In [22]:
results_log

NameError: name 'results_log' is not defined

In [13]:
import app

c:\Users\andrea.giorni\Desktop\code\Hugging face Agent course\Final_Assignment_Template\.venv310\lib\site-packages\gradio\oauth.py:162: UserWarning: Gradio does not support OAuth features outside of a Space environment. To help you debug your app locally, the login and logout buttons are mocked with your profile. To make it work, your machine must be logged in to Huggingface.
  warnings.warn(


ValueError: Your machine must be logged in to HF to debug a Gradio app locally. Please run `huggingface-cli login` or set `HF_TOKEN` as environment variable with one of your access token. You can generate a new token in your settings page (https://huggingface.co/settings/tokens).